In [45]:
rawUserArtistData = sc.textFile("user_artist_data.txt")

In [46]:
rawArtistData = sc.textFile("artist_data.txt")

In [47]:
def solve(line):
    if '\t' in line:
        identity, name = line.split('\t',1)
        try:
            ans = (int(identity), name.strip())
            return ans
        except ValueError, e:
            return None
    else:
        return None

In [49]:
artistByID = rawArtistData.map(lambda line: solve(line)).filter(lambda line: line!=None)

In [50]:
rawArtistAlias = sc.textFile("artist_alias.txt")

In [51]:
def solve2(line):
    tokens = line.split('\t')
    if tokens[0] == None:
        return None
    else:
        try:
            ans = (int(tokens[0]), int(tokens[1]))
            return ans
        except ValueError:
            return None

In [52]:
artistAlias = rawArtistAlias.map(solve2).filter(lambda line: line!=None).collectAsMap()

In [53]:
from pyspark.mllib.recommendation import ALS, Rating, MatrixFactorizationModel

In [54]:
bArtistAlias = sc.broadcast(artistAlias)

In [55]:
def solve3(line):
    userID, artistID, count = map(int, line.split(' '))
    ArtistAliasDict = bArtistAlias.value
    if artistID in ArtistAliasDict:
        finalArtistID = ArtistAliasDict[artistID]
    else:
        finalArtistID = artistID
    return Rating(userID, artistID, count)

In [56]:
trainData = rawUserArtistData.map(lambda line: solve3(line)).cache()

In [57]:
trainData.take(5)

[Rating(user=1000002, product=1, rating=55.0),
 Rating(user=1000002, product=1000006, rating=33.0),
 Rating(user=1000002, product=1000007, rating=8.0),
 Rating(user=1000002, product=1000009, rating=144.0),
 Rating(user=1000002, product=1000010, rating=314.0)]

In [58]:
model = ALS.trainImplicit(trainData, 10, 5, alpha=1.0)

In [59]:
rawArtistsForUser=rawUserArtistData.map(lambda line: line.split(' ')).filter(lambda line: int(line[0]) == 2093760)

In [60]:
rawArtistsForUser.take(10)

[[u'2093760', u'1180', u'1'],
 [u'2093760', u'1255340', u'3'],
 [u'2093760', u'378', u'1'],
 [u'2093760', u'813', u'2'],
 [u'2093760', u'942', u'7']]

In [61]:
existingProducts =set(rawArtistsForUser.map(lambda line: int(line[1])).collect())

In [62]:
recommendations = model.call("recommendProducts", 2093760, 10)

In [63]:
recommendedProducts = set([i[1] for i in recommendations])

In [64]:
artist_recommend = artistByID.filter(lambda line: line[0] in recommendedProducts).collect()
for i in artist_recommend:
    print(i[1])

50 Cent
Snoop Dogg
Jay-Z
Kanye West
Dr. Dre
Ludacris
2Pac
Eminem
The Game
Jay-Z and Linkin Park


In [65]:
artist_recommend

[(2814, u'50 Cent'),
 (4605, u'Snoop Dogg'),
 (1007614, u'Jay-Z'),
 (1037970, u'Kanye West'),
 (1811, u'Dr. Dre'),
 (1003249, u'Ludacris'),
 (1001819, u'2Pac'),
 (930, u'Eminem'),
 (1300642, u'The Game'),
 (6914803, u'Jay-Z and Linkin Park')]